In [1]:
import sqlalchemy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

database = 'qardio'
host = '127.0.0.1'
user = 'root'
password = 'Party100'
url = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = sqlalchemy.create_engine(url, echo=True)
conn = engine.connect()

2023-06-22 14:13:57,175 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-06-22 14:13:57,176 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-22 14:13:57,180 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-06-22 14:13:57,185 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-22 14:13:57,193 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-06-22 14:13:57,196 INFO sqlalchemy.engine.Engine [raw sql] {}


In [ ]:
# query database
existing_data = pd.read_sql(sql='SELECT * FROM day_email_rev;', con=conn)